In [13]:
import pandas as pd
import seaborn as sns

In [23]:
data=pd.read_hdf("model1.h5")
data.shape

(189522, 4)

In [24]:
train_perc = 0.8
test_split = 0.2
 
train_split = int(len(data) * train_perc)
train_data = data[:train_split]
test_split = int(len(data) * test_split)
test_data = data[train_split:(train_split+test_split+1)]

In [25]:
mask2 = (train_data['mill_dp_mbar_pv']>25) & (train_data['mill_dp_mbar_pv']<45) & (train_data['separator_speed_rpm_pv']>600) & (train_data['separator_speed_rpm_pv']<850) & (train_data['separator_pwr_kw_pv']<62)

In [26]:
train_data = train_data[mask2].reset_index(drop=True)

In [27]:
train_data['date'] = pd.to_datetime(train_data['date'], dayfirst=True)
train_data['date'] = train_data['date'].apply(lambda x: x.replace(second=0))
train_data = train_data.groupby(train_data['date']).mean()

In [28]:
train_data

,separator_speed_rpm_pv,mill_dp_mbar_pv,separator_pwr_kw_pv
date,,,
2019-05-10 11:24:00,726.700,30.150,58.085
2019-05-10 11:25:00,742.145,32.660,55.975
2019-05-10 11:26:00,747.585,34.695,55.540
2019-05-10 11:27:00,747.380,35.065,55.535
2019-05-10 11:28:00,747.190,35.490,55.980
...,...,...,...
2019-09-29 15:44:00,797.495,36.330,57.245
2019-09-29 15:45:00,794.880,36.200,57.590
2019-09-29 15:46:00,795.305,36.335,57.255


In [29]:
from statsmodels.graphics.gofplots import qqplot
from matplotlib import pyplot
from scipy import stats 
from statsmodels.stats import diagnostic


In [33]:
train_data = train_data.loc['2019-05-10 11:24:00':, :]

In [34]:
train_data_freq=train_data.asfreq("1T")

In [35]:
train_data_freq = train_data_freq.interpolate(method='linear', axis=0, limit=None, inplace=False, limit_direction='backward', limit_area="inside" )

In [36]:
def df_derived_by_shift(df,lag,NON_DER=[]):
    df = df.copy()
    if not lag:
        return df
    cols ={}
    for i in range(1,lag+1):
        for x in list(df.columns):
            if x not in NON_DER:
                if not x in cols:
                    cols[x] = ['{}_{}'.format(x, i)]
                else:
                    cols[x].append('{}_{}'.format(x, i))
    for k,v in cols.items():
        columns = v
        dfn = pd.DataFrame(data=None, columns=columns, index=df.index)    
        i = 1
        for c in columns:
            dfn[c] = df[k].shift(periods=+i)
            i+=1
        df = pd.concat([df, dfn], axis=1, join_axes=[df.index])
    return df

In [55]:
lag1=ms_inter[["separator_pwr_kw_pv","separator_speed_rpm_pv"]]

lag2=ms_inter[["separator_pwr_kw_pv", "mill_dp_mbar_pv"]]

In [50]:
lag1_new=df_derived_by_shift(lag1,10,NON_DER=["separator_pwr_kw_pv"]).dropna()

c:\users\kostantina\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:20: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.


In [51]:
lag1_new

,separator_pwr_kw_pv,separator_speed_rpm_pv,separator_speed_rpm_pv_1,separator_speed_rpm_pv_2,separator_speed_rpm_pv_3,separator_speed_rpm_pv_4,separator_speed_rpm_pv_5,separator_speed_rpm_pv_6,separator_speed_rpm_pv_7,separator_speed_rpm_pv_8,separator_speed_rpm_pv_9,separator_speed_rpm_pv_10
date,,,,,,,,,,,,
2019-05-10 21:35:00,56.175,748.275,747.455,748.325,717.590,717.065,701.485,682.520,682.760,683.525,682.625,682.945
2019-05-10 21:36:00,56.015,747.165,748.275,747.455,748.325,717.590,717.065,701.485,682.520,682.760,683.525,682.625
2019-05-10 21:37:00,56.095,747.980,747.165,748.275,747.455,748.325,717.590,717.065,701.485,682.520,682.760,683.525
2019-05-10 21:38:00,56.000,747.375,747.980,747.165,748.275,747.455,748.325,717.590,717.065,701.485,682.520,682.760
2019-05-10 21:39:00,56.050,747.640,747.375,747.980,747.165,748.275,747.455,748.325,717.590,717.065,701.485,682.520
...,...,...,...,...,...,...,...,...,...,...,...,...
2019-09-29 15:44:00,57.245,797.495,795.410,796.360,795.490,795.225,795.275,795.220,797.680,795.225,795.830,795.360
2019-09-29 15:45:00,57.590,794.880,797.495,795.410,796.360,795.490,795.225,795.275,795.220,797.680,795.225,795.830
2019-09-29 15:46:00,57.255,795.305,794.880,797.495,795.410,796.360,795.490,795.225,795.275,795.220,797.680,795.225


In [52]:
lag1_new.corr()

,separator_pwr_kw_pv,separator_speed_rpm_pv,separator_speed_rpm_pv_1,separator_speed_rpm_pv_2,separator_speed_rpm_pv_3,separator_speed_rpm_pv_4,separator_speed_rpm_pv_5,separator_speed_rpm_pv_6,separator_speed_rpm_pv_7,separator_speed_rpm_pv_8,separator_speed_rpm_pv_9,separator_speed_rpm_pv_10
separator_pwr_kw_pv,1.000000,0.558645,0.560188,0.555425,0.550997,0.546340,0.540761,0.535452,0.527615,0.517156,0.502912,0.486091
separator_speed_rpm_pv,0.558645,1.000000,0.979080,0.958307,0.940919,0.925017,0.909919,0.894485,0.876437,0.855346,0.828303,0.798243
separator_speed_rpm_pv_1,0.560188,0.979080,1.000000,0.980131,0.960412,0.942974,0.926280,0.909709,0.890626,0.868317,0.840953,0.810379
separator_speed_rpm_pv_2,0.555425,0.958307,0.980131,1.000000,0.980983,0.961159,0.943103,0.925102,0.905002,0.881710,0.853353,0.822550
separator_speed_rpm_pv_3,0.550997,0.940919,0.960412,0.980983,1.000000,0.981031,0.961030,0.942048,0.920818,0.896844,0.867647,0.835911
separator_speed_rpm_pv_4,0.546340,0.925017,0.942974,0.961159,0.981031,1.000000,0.981110,0.960514,0.938759,0.913917,0.884171,0.852039
separator_speed_rpm_pv_5,0.540761,0.909919,0.926280,0.943103,0.961030,0.981110,1.000000,0.980839,0.958046,0.933157,0.903008,0.870353
separator_speed_rpm_pv_6,0.535452,0.894485,0.909709,0.925102,0.942048,0.960514,0.980839,1.000000,0.979639,0.954632,0.925000,0.892284
separator_speed_rpm_pv_7,0.527615,0.876437,0.890626,0.905002,0.920818,0.938759,0.958046,0.979639,1.000000,0.978738,0.950063,0.917996
separator_speed_rpm_pv_8,0.517156,0.855346,0.868317,0.881710,0.896844,0.913917,0.933157,0.954632,0.978738,1.000000,0.976703,0.946142


In [53]:
lag1_new.corr(method = "spearman")

,separator_pwr_kw_pv,separator_speed_rpm_pv,separator_speed_rpm_pv_1,separator_speed_rpm_pv_2,separator_speed_rpm_pv_3,separator_speed_rpm_pv_4,separator_speed_rpm_pv_5,separator_speed_rpm_pv_6,separator_speed_rpm_pv_7,separator_speed_rpm_pv_8,separator_speed_rpm_pv_9,separator_speed_rpm_pv_10
separator_pwr_kw_pv,1.000000,0.551672,0.552858,0.549669,0.546941,0.543699,0.540318,0.536990,0.532501,0.527195,0.521774,0.516273
separator_speed_rpm_pv,0.551672,1.000000,0.989712,0.979843,0.970448,0.960961,0.951458,0.941756,0.931287,0.920550,0.909415,0.898356
separator_speed_rpm_pv_1,0.552858,0.989712,1.000000,0.989735,0.979680,0.969934,0.960058,0.949852,0.939126,0.927774,0.916345,0.904934
separator_speed_rpm_pv_2,0.549669,0.979843,0.989735,1.000000,0.989639,0.979283,0.969217,0.958703,0.947498,0.935893,0.923830,0.912113
separator_speed_rpm_pv_3,0.546941,0.970448,0.979680,0.989639,1.000000,0.989461,0.978854,0.968223,0.956702,0.944629,0.932327,0.920008
separator_speed_rpm_pv_4,0.543699,0.960961,0.969934,0.979283,0.989461,1.000000,0.989366,0.978313,0.966732,0.954306,0.941578,0.929090
separator_speed_rpm_pv_5,0.540318,0.951458,0.960058,0.969217,0.978854,0.989366,1.000000,0.989082,0.977263,0.964765,0.951751,0.938690
separator_speed_rpm_pv_6,0.536990,0.941756,0.949852,0.958703,0.968223,0.978313,0.989082,1.000000,0.988634,0.975994,0.963012,0.949618
separator_speed_rpm_pv_7,0.532501,0.931287,0.939126,0.947498,0.956702,0.966732,0.977263,0.988634,1.000000,0.988190,0.975180,0.961672
separator_speed_rpm_pv_8,0.527195,0.920550,0.927774,0.935893,0.944629,0.954306,0.964765,0.975994,0.988190,1.000000,0.987915,0.974372


In [59]:
lag2_new=df_derived_by_shift(lag2,10,NON_DER=["separator_pwr_kw_pv"])
lag2_new.corr()

c:\users\kostantina\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:20: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.


,separator_pwr_kw_pv,mill_dp_mbar_pv,mill_dp_mbar_pv_1,mill_dp_mbar_pv_2,mill_dp_mbar_pv_3,mill_dp_mbar_pv_4,mill_dp_mbar_pv_5,mill_dp_mbar_pv_6,mill_dp_mbar_pv_7,mill_dp_mbar_pv_8,mill_dp_mbar_pv_9,mill_dp_mbar_pv_10
separator_pwr_kw_pv,1.000000,0.457374,0.452528,0.445275,0.434165,0.421606,0.409074,0.396090,0.386854,0.376785,0.368732,0.361758
mill_dp_mbar_pv,0.457374,1.000000,0.965540,0.919369,0.881998,0.853479,0.831464,0.813768,0.798263,0.783869,0.771755,0.760557
mill_dp_mbar_pv_1,0.452528,0.965540,1.000000,0.965540,0.919369,0.881998,0.853478,0.831463,0.813768,0.798262,0.783869,0.771755
mill_dp_mbar_pv_2,0.445275,0.919369,0.965540,1.000000,0.965540,0.919369,0.881998,0.853478,0.831463,0.813767,0.798262,0.783869
mill_dp_mbar_pv_3,0.434165,0.881998,0.919369,0.965540,1.000000,0.965540,0.919369,0.881997,0.853478,0.831463,0.813767,0.798262
mill_dp_mbar_pv_4,0.421606,0.853479,0.881998,0.919369,0.965540,1.000000,0.965540,0.919369,0.881997,0.853478,0.831463,0.813767
mill_dp_mbar_pv_5,0.409074,0.831464,0.853478,0.881998,0.919369,0.965540,1.000000,0.965540,0.919369,0.881997,0.853477,0.831463
mill_dp_mbar_pv_6,0.396090,0.813768,0.831463,0.853478,0.881997,0.919369,0.965540,1.000000,0.965540,0.919369,0.881997,0.853477
mill_dp_mbar_pv_7,0.386854,0.798263,0.813768,0.831463,0.853478,0.881997,0.919369,0.965540,1.000000,0.965540,0.919369,0.881997
mill_dp_mbar_pv_8,0.376785,0.783869,0.798262,0.813767,0.831463,0.853478,0.881997,0.919369,0.965540,1.000000,0.965541,0.919369
